In [ ]:
!pip install sentence-transformers
!pip install llama-index
!pip install langchain
!pip install faiss-gpu
!pip install icetk
!pip install cpm_kernels

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install protobuf
!pip install --upgrade protobuf

In [ ]:
import faiss
from llama_index import SimpleDirectoryReader, LangchainEmbedding, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index.node_parser import SimpleNodeParser

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [ ]:
text_splitter = CharacterTextSplitter(separator='\n\n', chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('/content/drive/MyDrive/Colab Notebooks/data/faq').load_data()
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('THUDM/chatglm-6b-int4', trust_remote_code=True)
model = AutoModel.from_pretrained('THUDM/chatglm-6b-int4', trust_remote_code=True).half().cuda()
model = model.eval()

In [ ]:
from langchain.llms.base import LLM

class CustomLLM(LLM):
  def _call(self, prompt, stop = None):
    response, history = model.chat(tokenizer, prompt, history=[])
    return response
  
  @property
  def _identifying_params(self):
    return {'name_of_model': 'chatglm-6b-int4'}

  @property
  def _llm_type(self):
    return 'custom'

In [ ]:
from llama_index import LLMPredictor

llm_predictor = LLMPredictor(llm=CustomLLM())

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
        model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
))
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

In [ ]:
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display
from llama_index import StorageContext, GPTVectorStoreIndex

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)

In [ ]:
from llama_index import QuestionAnswerPrompt

QA_PROMPT_TMPL = (
    "{context_str}"
    "\n\n"
    "根据以上信息，请回答下面的问题：\n"
    "Q: {query_str}\n"
)

QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

query_engine = index.as_query_engine(retriever_mode='embedding', verbose=True, text_qa_template=QA_PROMPT)

In [ ]:
response = query_engine.query("请问你们海南能发货吗？")

print(response)

A: 我们支持全国大部分省份的配送，包括海南。
